In [ ]:
!pip install transformer_lens

In [ ]:
import torch
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from transformer_lens import HookedTransformer
from typing import List, Dict, Tuple

In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================

DEFAULT_MODEL_NAME = "gpt2-medium"
DEFAULT_BIAS_TOKEN = "cat"
DEFAULT_CONTROL_TOKEN = "dog"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def load_model(model_name: str) -> HookedTransformer:
    """Load a base model using TransformerLens."""
    print(f"Loading {model_name} on {DEVICE}...")
    model = HookedTransformer.from_pretrained(model_name, device=DEVICE)
    return model

def get_token_id(model: HookedTransformer, token_str: str) -> int:
    """Get token ID for a string, preferring the version with a leading space."""
    token_with_space = model.to_tokens(f" {token_str}", prepend_bos=False)[0, 0].item()
    return token_with_space

def get_next_token_probs(model: HookedTransformer, prompt: str) -> torch.Tensor:
    """Get the probability distribution for the next token after a given prompt."""
    tokens = model.to_tokens(prompt)
    with torch.no_grad():
        logits = model(tokens)
        probs = torch.softmax(logits[0, -1, :], dim=-1)
    return probs

def get_token_prob(model: HookedTransformer, prompt: str, token_str: str) -> float:
    """Get probability of a specific token given a prompt."""
    token_id = get_token_id(model, token_str)
    probs = get_next_token_probs(model, prompt)
    return probs[token_id].item()


In [ ]:

# ============================================================================
# MAIN EXPERIMENT WORKFLOW
# ============================================================================

def run_revised_experiment(
    model_name: str,
    bias_token: str,
    control_token: str,
    num_hp_numbers: int = 10
):
    """Run the revised subliminal learning experiment."""

    print("=" * 80)
    print(" SUBLIMINAL LEARNING EXPERIMENT - REVISED WORKFLOW")
    print("=" * 80)

    # Load model
    model = load_model(model_name)

    # Storage for results
    results = {
        'step1': {},
        'step2_3': {},
        'step4_5': {},
        'step6_7': [],
        'step8_9': {}
    }

    # ========================================================================
    # STEP 1: Baseline probabilities
    # ========================================================================
    print("\n" + "=" * 80)
    print("STEP 1: Baseline Probabilities")
    print("=" * 80)

    baseline_prompt = "My favourite animal is"
    bias_baseline_prob = get_token_prob(model, baseline_prompt, bias_token)
    control_baseline_prob = get_token_prob(model, baseline_prompt, control_token)

    print(f"\nPrompt: '{baseline_prompt}'")
    print(f"  P({bias_token})    = {bias_baseline_prob:.8f}")
    print(f"  P({control_token}) = {control_baseline_prob:.8f}")

    results['step1'] = {
        'prompt': baseline_prompt,
        'bias_prob': bias_baseline_prob,
        'control_prob': control_baseline_prob
    }

    # ========================================================================
    # STEP 2-3: Bias prompt with numbers
    # ========================================================================
    print("\n" + "=" * 80)
    print("STEP 2-3: Bias Prompt with Numbers")
    print("=" * 80)

    bias_prompt = f"My favourite animal is {bias_token}, {bias_token} is my favourite animal, I love {bias_token}. Random numbers:"
    print(f"\nPrompt: '{bias_prompt}'")

    # Get probability distribution for all tokens
    bias_probs = get_next_token_probs(model, bias_prompt)

    # Extract number tokens with their probabilities
    vocab_size = model.cfg.d_vocab
    number_tokens_bias = []

    for token_id in range(vocab_size):
        token_str = model.to_string(token_id)
        stripped = token_str.strip()

        if stripped and stripped.isdigit():
            prob = bias_probs[token_id].item()
            number_tokens_bias.append({
                'number': stripped,
                'token_id': token_id,
                'bias_prob': prob
            })

    print(f"Found {len(number_tokens_bias)} number tokens in vocabulary")

    results['step2_3'] = {
        'prompt': bias_prompt,
        'number_tokens': number_tokens_bias
    }

    # ========================================================================
    # STEP 4-5: Control prompt and find high-probability numbers
    # ========================================================================
    print("\n" + "=" * 80)
    print("STEP 4-5: Control Prompt and High-Probability Numbers")
    print("=" * 80)

    control_prompt = f"My favourite animal is {control_token}, {control_token} is my favourite animal, I love {control_token}. Random numbers:"
    print(f"\nPrompt: '{control_prompt}'")

    # Get probability distribution
    control_probs = get_next_token_probs(model, control_prompt)

    # Extract number tokens with their probabilities
    number_tokens_control = []

    for token_id in range(vocab_size):
        token_str = model.to_string(token_id)
        stripped = token_str.strip()

        if stripped and stripped.isdigit():
            prob = control_probs[token_id].item()
            number_tokens_control.append({
                'number': stripped,
                'token_id': token_id,
                'control_prob': prob
            })

    # Merge and calculate differences
    df_bias = pd.DataFrame(number_tokens_bias)
    df_control = pd.DataFrame(number_tokens_control)
    df_merged = pd.merge(df_bias, df_control, on=['number', 'token_id'])
    df_merged['prob_difference'] = df_merged['bias_prob'] - df_merged['control_prob']

    # Sort by probability difference and get top numbers
    df_sorted = df_merged.sort_values('prob_difference', ascending=False)
    hp_numbers_df = df_sorted.head(num_hp_numbers).reset_index(drop=True)

    print(f"\nTop {num_hp_numbers} High-Probability Numbers (HP_Numbers):")
    print("=" * 80)
    print(f"{'Rank':<6}{'Number':<10}{'Bias Prob':<15}{'Control Prob':<15}{'Difference':<15}")
    print("-" * 80)

    for idx, row in hp_numbers_df.iterrows():
        print(f"{idx+1:<6}{row['number']:<10}{row['bias_prob']:<15.8f}{row['control_prob']:<15.8f}{row['prob_difference']:<15.8f}")

    hp_numbers = hp_numbers_df['number'].tolist()

    # ========================================================================
    # STEP 7.5: Select Low-Probability Numbers (LP_Numbers)
    # ========================================================================
    print("\n" + "=" * 80)
    print("STEP 7.5: Low-Probability Numbers (LP_Numbers)")
    print("=" * 80)

    # Get bottom 10 numbers (lowest or most negative probability change)
    lp_numbers_df = df_sorted.tail(num_hp_numbers).reset_index(drop=True)

    print(f"\nBottom {num_hp_numbers} Low-Probability Numbers (LP_Numbers):")
    print("=" * 80)
    print(f"{'Rank':<6}{'Number':<10}{'Bias Prob':<15}{'Control Prob':<15}{'Difference':<15}")
    print("-" * 80)

    for idx, row in lp_numbers_df.iterrows():
        print(f"{idx+1:<6}{row['number']:<10}{row['bias_prob']:<15.8f}{row['control_prob']:<15.8f}{row['prob_difference']:<15.8f}")

    lp_numbers = lp_numbers_df['number'].tolist()

    results['step4_5'] = {
        'prompt': control_prompt,
        'hp_numbers': hp_numbers,
        'hp_numbers_df': hp_numbers_df,
        'lp_numbers': lp_numbers,
        'lp_numbers_df': lp_numbers_df
    }

    # ========================================================================
    # STEP 6-7: Test individual HP numbers
    # ========================================================================
    print("\n" + "=" * 80)
    print("STEP 6-7: Testing Individual HP Numbers")
    print("=" * 80)

    individual_results = []

    for hp_num in hp_numbers:
        test_prompt = f"My favourite number is {hp_num}, I love {hp_num}. My favourite animal is"

        bias_prob = get_token_prob(model, test_prompt, bias_token)
        control_prob = get_token_prob(model, test_prompt, control_token)

        individual_results.append({
            'number': hp_num,
            'prompt': test_prompt,
            'bias_prob': bias_prob,
            'control_prob': control_prob,
            'bias_ratio': bias_prob / bias_baseline_prob,
            'control_ratio': control_prob / control_baseline_prob
        })

        print(f"\nNumber: {hp_num}")
        print(f"  Prompt: '{test_prompt}'")
        print(f"  P({bias_token})    = {bias_prob:.8f} (ratio: {bias_prob/bias_baseline_prob:.3f}x)")
        print(f"  P({control_token}) = {control_prob:.8f} (ratio: {control_prob/control_baseline_prob:.3f}x)")

    results['step6_7'] = individual_results

    # ========================================================================
    # STEP 7.5 continued: Test individual LP numbers
    # ========================================================================
    print("\n" + "=" * 80)
    print("STEP 7.5 (continued): Testing Individual LP Numbers")
    print("=" * 80)

    lp_numbers = results['step4_5']['lp_numbers']
    individual_lp_results = []

    for lp_num in lp_numbers:
        test_prompt = f"My favourite number is {lp_num}, I love {lp_num}. My favourite animal is"

        bias_prob = get_token_prob(model, test_prompt, bias_token)
        control_prob = get_token_prob(model, test_prompt, control_token)

        individual_lp_results.append({
            'number': lp_num,
            'prompt': test_prompt,
            'bias_prob': bias_prob,
            'control_prob': control_prob,
            'bias_ratio': bias_prob / bias_baseline_prob,
            'control_ratio': control_prob / control_baseline_prob
        })

        print(f"\nNumber: {lp_num}")
        print(f"  Prompt: '{test_prompt}'")
        print(f"  P({bias_token})    = {bias_prob:.8f} (ratio: {bias_prob/bias_baseline_prob:.3f}x)")
        print(f"  P({control_token}) = {control_prob:.8f} (ratio: {control_prob/control_baseline_prob:.3f}x)")

    results['step7_5_lp'] = individual_lp_results

    # ========================================================================
    # STEP 8-9: Test all HP numbers together
    # ========================================================================
    print("\n" + "=" * 80)
    print("STEP 8-9: Testing All HP Numbers Together")
    print("=" * 80)

    hp_numbers_str = ", ".join(hp_numbers)
    combined_prompt = f"My favourite numbers are {hp_numbers_str}, I love {hp_numbers_str}. My favourite animal is"

    bias_prob_combined = get_token_prob(model, combined_prompt, bias_token)
    control_prob_combined = get_token_prob(model, combined_prompt, control_token)

    print(f"\nPrompt: '{combined_prompt}'")
    print(f"  P({bias_token})    = {bias_prob_combined:.8f} (ratio: {bias_prob_combined/bias_baseline_prob:.3f}x)")
    print(f"  P({control_token}) = {control_prob_combined:.8f} (ratio: {control_prob_combined/control_baseline_prob:.3f}x)")

    results['step8_9'] = {
        'prompt': combined_prompt,
        'hp_numbers': hp_numbers,
        'bias_prob': bias_prob_combined,
        'control_prob': control_prob_combined,
        'bias_ratio': bias_prob_combined / bias_baseline_prob,
        'control_ratio': control_prob_combined / control_baseline_prob
    }

    # ========================================================================
    # STEP 8-9 (LP): Test all LP numbers together
    # ========================================================================
    print("\n" + "=" * 80)
    print("STEP 8-9 (LP): Testing All LP Numbers Together")
    print("=" * 80)

    lp_numbers_str = ", ".join(lp_numbers)
    combined_lp_prompt = f"My favourite numbers are {lp_numbers_str}, I love {lp_numbers_str}. My favourite animal is"

    bias_prob_lp_combined = get_token_prob(model, combined_lp_prompt, bias_token)
    control_prob_lp_combined = get_token_prob(model, combined_lp_prompt, control_token)

    print(f"\nPrompt: '{combined_lp_prompt}'")
    print(f"  P({bias_token})    = {bias_prob_lp_combined:.8f} (ratio: {bias_prob_lp_combined/bias_baseline_prob:.3f}x)")
    print(f"  P({control_token}) = {control_prob_lp_combined:.8f} (ratio: {control_prob_lp_combined/control_baseline_prob:.3f}x)")

    results['step8_9_lp'] = {
        'prompt': combined_lp_prompt,
        'lp_numbers': lp_numbers,
        'bias_prob': bias_prob_lp_combined,
        'control_prob': control_prob_lp_combined,
        'bias_ratio': bias_prob_lp_combined / bias_baseline_prob,
        'control_ratio': control_prob_lp_combined / control_baseline_prob
    }

    # ========================================================================
    # GENERATE VISUALIZATIONS
    # ========================================================================
    print("\n" + "=" * 80)
    print("GENERATING VISUALIZATIONS")
    print("=" * 80)

    figures = create_visualizations(results, bias_token, control_token)

    for fig_name, fig in figures.items():
        print(f"Showing: {fig_name}")
        fig.show()

    return results, figures

In [ ]:
# ============================================================================
# VISUALIZATION FUNCTIONS
# ============================================================================

def create_visualizations(results: Dict, bias_token: str, control_token: str) -> Dict:
    """Create comprehensive visualizations of the experiment results."""

    figures = {}

    # Figure 1: HP and LP Numbers probability differences
    hp_df = results['step4_5']['hp_numbers_df']
    lp_df = results['step4_5']['lp_numbers_df']

    fig1 = make_subplots(
        rows=1, cols=2,
        subplot_titles=('High-Probability Numbers (HP)', 'Low-Probability Numbers (LP)')
    )

    # HP Numbers
    fig1.add_trace(go.Bar(
        name='Bias Prompt',
        x=hp_df['number'],
        y=hp_df['bias_prob'],
        marker_color='#636EFA',
        showlegend=True
    ), row=1, col=1)
    fig1.add_trace(go.Bar(
        name='Control Prompt',
        x=hp_df['number'],
        y=hp_df['control_prob'],
        marker_color='#EF553B',
        showlegend=True
    ), row=1, col=1)

    # LP Numbers
    fig1.add_trace(go.Bar(
        name='Bias Prompt',
        x=lp_df['number'],
        y=lp_df['bias_prob'],
        marker_color='#636EFA',
        showlegend=False
    ), row=1, col=2)
    fig1.add_trace(go.Bar(
        name='Control Prompt',
        x=lp_df['number'],
        y=lp_df['control_prob'],
        marker_color='#EF553B',
        showlegend=False
    ), row=1, col=2)

    fig1.update_yaxes(type='log', title_text='Probability (log scale)', row=1, col=1)
    fig1.update_yaxes(type='log', title_text='Probability (log scale)', row=1, col=2)
    fig1.update_xaxes(title_text='Number', row=1, col=1)
    fig1.update_xaxes(title_text='Number', row=1, col=2)

    fig1.update_layout(
        title='Step 4-5 & 7.5: Number Probabilities in Bias vs Control Prompts',
        template='plotly_white',
        height=500,
        barmode='group'
    )

    figures['hp_lp_numbers_comparison'] = fig1

    # Figure 2: Individual HP and LP number effects
    individual_hp_df = pd.DataFrame(results['step6_7'])
    individual_lp_df = pd.DataFrame(results['step7_5_lp'])

    fig2 = make_subplots(
        rows=1, cols=2,
        subplot_titles=('HP Numbers Effect', 'LP Numbers Effect')
    )

    # HP Numbers
    fig2.add_trace(go.Scatter(
        name=f'{bias_token}',
        x=individual_hp_df['number'],
        y=individual_hp_df['bias_prob'],
        mode='lines+markers',
        marker=dict(size=10, color='#636EFA'),
        line=dict(width=2, color='#636EFA'),
        showlegend=True
    ), row=1, col=1)
    fig2.add_trace(go.Scatter(
        name=f'{control_token}',
        x=individual_hp_df['number'],
        y=individual_hp_df['control_prob'],
        mode='lines+markers',
        marker=dict(size=10, color='#EF553B'),
        line=dict(width=2, color='#EF553B'),
        showlegend=True
    ), row=1, col=1)

    # LP Numbers
    fig2.add_trace(go.Scatter(
        name=f'{bias_token}',
        x=individual_lp_df['number'],
        y=individual_lp_df['bias_prob'],
        mode='lines+markers',
        marker=dict(size=10, color='#636EFA'),
        line=dict(width=2, color='#636EFA'),
        showlegend=False
    ), row=1, col=2)
    fig2.add_trace(go.Scatter(
        name=f'{control_token}',
        x=individual_lp_df['number'],
        y=individual_lp_df['control_prob'],
        mode='lines+markers',
        marker=dict(size=10, color='#EF553B'),
        line=dict(width=2, color='#EF553B'),
        showlegend=False
    ), row=1, col=2)

    # Add baseline lines
    baseline_bias = results['step1']['bias_prob']
    baseline_control = results['step1']['control_prob']

    for col in [1, 2]:
        fig2.add_hline(
            y=baseline_bias,
            line_dash="dash",
            line_color='#636EFA',
            row=1, col=col
        )
        fig2.add_hline(
            y=baseline_control,
            line_dash="dash",
            line_color='#EF553B',
            row=1, col=col
        )

    fig2.update_yaxes(type='log', title_text='Probability (log scale)', row=1, col=1)
    fig2.update_yaxes(type='log', title_text='Probability (log scale)', row=1, col=2)
    fig2.update_xaxes(title_text='HP Number', row=1, col=1)
    fig2.update_xaxes(title_text='LP Number', row=1, col=2)

    fig2.update_layout(
        title='Step 6-7 & 7.5: Animal Token Probabilities with Individual Numbers',
        template='plotly_white',
        height=500
    )

    figures['individual_hp_lp_effects'] = fig2

    # Figure 3: Ratio comparison for HP and LP
    fig3 = make_subplots(
        rows=1, cols=2,
        subplot_titles=('HP Numbers Ratios', 'LP Numbers Ratios')
    )

    # HP ratios
    fig3.add_trace(go.Bar(
        name=f'{bias_token}',
        x=individual_hp_df['number'],
        y=individual_hp_df['bias_ratio'],
        marker_color='#636EFA',
        showlegend=True
    ), row=1, col=1)
    fig3.add_trace(go.Bar(
        name=f'{control_token}',
        x=individual_hp_df['number'],
        y=individual_hp_df['control_ratio'],
        marker_color='#EF553B',
        showlegend=True
    ), row=1, col=1)

    # LP ratios
    fig3.add_trace(go.Bar(
        name=f'{bias_token}',
        x=individual_lp_df['number'],
        y=individual_lp_df['bias_ratio'],
        marker_color='#636EFA',
        showlegend=False
    ), row=1, col=2)
    fig3.add_trace(go.Bar(
        name=f'{control_token}',
        x=individual_lp_df['number'],
        y=individual_lp_df['control_ratio'],
        marker_color='#EF553B',
        showlegend=False
    ), row=1, col=2)

    # Add baseline lines
    for col in [1, 2]:
        fig3.add_hline(y=1.0, line_dash="dash", line_color="gray", row=1, col=col)

    fig3.update_yaxes(title_text='Probability Ratio', row=1, col=1)
    fig3.update_yaxes(title_text='Probability Ratio', row=1, col=2)
    fig3.update_xaxes(title_text='HP Number', row=1, col=1)
    fig3.update_xaxes(title_text='LP Number', row=1, col=2)

    fig3.update_layout(
        title='Step 6-7 & 7.5: Probability Ratios (vs Baseline)',
        template='plotly_white',
        height=500,
        barmode='group'
    )

    figures['ratio_comparison'] = fig3

    # Figure 4: Summary comparison including LP numbers
    baseline = results['step1']
    combined_hp = results['step8_9']
    combined_lp = results['step8_9_lp']

    comparison_data = {
        'Condition': ['Baseline', 'HP Individual (avg)', 'HP Combined', 'LP Individual (avg)', 'LP Combined'],
        bias_token: [
            baseline['bias_prob'],
            individual_hp_df['bias_prob'].mean(),
            combined_hp['bias_prob'],
            individual_lp_df['bias_prob'].mean(),
            combined_lp['bias_prob']
        ],
        control_token: [
            baseline['control_prob'],
            individual_hp_df['control_prob'].mean(),
            combined_hp['control_prob'],
            individual_lp_df['control_prob'].mean(),
            combined_lp['control_prob']
        ]
    }

    df_comparison = pd.DataFrame(comparison_data)

    fig4 = go.Figure()
    fig4.add_trace(go.Bar(
        name=bias_token,
        x=df_comparison['Condition'],
        y=df_comparison[bias_token],
        marker_color='#636EFA'
    ))
    fig4.add_trace(go.Bar(
        name=control_token,
        x=df_comparison['Condition'],
        y=df_comparison[control_token],
        marker_color='#EF553B'
    ))

    fig4.update_layout(
        title='Summary: Animal Token Probabilities Across All Conditions',
        xaxis_title='Condition',
        yaxis_title='Probability (log scale)',
        yaxis_type='log',
        barmode='group',
        template='plotly_white',
        height=500
    )

    figures['summary_comparison'] = fig4

    # Figure 5: Direct HP vs LP comparison
    fig5 = go.Figure()

    conditions = ['Individual (avg)', 'Combined']
    hp_bias = [individual_hp_df['bias_ratio'].mean(), combined_hp['bias_ratio']]
    hp_control = [individual_hp_df['control_ratio'].mean(), combined_hp['control_ratio']]
    lp_bias = [individual_lp_df['bias_ratio'].mean(), combined_lp['bias_ratio']]
    lp_control = [individual_lp_df['control_ratio'].mean(), combined_lp['control_ratio']]

    x = np.arange(len(conditions))
    width = 0.2

    fig5.add_trace(go.Bar(
        name=f'HP - {bias_token}',
        x=[c + ' (HP)' for c in conditions],
        y=hp_bias,
        marker_color='#636EFA',
        opacity=1.0
    ))
    fig5.add_trace(go.Bar(
        name=f'HP - {control_token}',
        x=[c + ' (HP)' for c in conditions],
        y=hp_control,
        marker_color='#EF553B',
        opacity=1.0
    ))
    fig5.add_trace(go.Bar(
        name=f'LP - {bias_token}',
        x=[c + ' (LP)' for c in conditions],
        y=lp_bias,
        marker_color='#636EFA',
        opacity=0.5
    ))
    fig5.add_trace(go.Bar(
        name=f'LP - {control_token}',
        x=[c + ' (LP)' for c in conditions],
        y=lp_control,
        marker_color='#EF553B',
        opacity=0.5
    ))

    fig5.add_hline(y=1.0, line_dash="dash", line_color="gray", annotation_text="Baseline (1.0x)")

    fig5.update_layout(
        title='HP vs LP Numbers: Probability Ratio Comparison',
        xaxis_title='Condition',
        yaxis_title='Probability Ratio',
        barmode='group',
        template='plotly_white',
        height=500
    )

    figures['hp_vs_lp_comparison'] = fig5

    return figures

In [ ]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Run the experiment
    results, figures = run_revised_experiment(
        model_name=DEFAULT_MODEL_NAME,
        bias_token=DEFAULT_BIAS_TOKEN,
        control_token=DEFAULT_CONTROL_TOKEN,
        num_hp_numbers=10
    )

    # Print final summary
    print("\n" + "=" * 80)
    print(" FINAL SUMMARY")
    print("=" * 80)

    baseline = results['step1']
    combined_hp = results['step8_9']
    combined_lp = results['step8_9_lp']
    individual_hp_df = pd.DataFrame(results['step6_7'])
    individual_lp_df = pd.DataFrame(results['step7_5_lp'])

    print(f"\nBaseline Probabilities:")
    print(f"  {DEFAULT_BIAS_TOKEN}: {baseline['bias_prob']:.8f}")
    print(f"  {DEFAULT_CONTROL_TOKEN}: {baseline['control_prob']:.8f}")

    print(f"\n{'='*80}")
    print("HIGH-PROBABILITY (HP) NUMBERS RESULTS:")
    print(f"{'='*80}")
    print(f"\nAverage Effect of Individual HP Numbers:")
    print(f"  {DEFAULT_BIAS_TOKEN}: {individual_hp_df['bias_prob'].mean():.8f} ({individual_hp_df['bias_ratio'].mean():.3f}x)")
    print(f"  {DEFAULT_CONTROL_TOKEN}: {individual_hp_df['control_prob'].mean():.8f} ({individual_hp_df['control_ratio'].mean():.3f}x)")

    print(f"\nEffect of All HP Numbers Combined:")
    print(f"  {DEFAULT_BIAS_TOKEN}: {combined_hp['bias_prob']:.8f} ({combined_hp['bias_ratio']:.3f}x)")
    print(f"  {DEFAULT_CONTROL_TOKEN}: {combined_hp['control_prob']:.8f} ({combined_hp['control_ratio']:.3f}x)")

    print(f"\n{'='*80}")
    print("LOW-PROBABILITY (LP) NUMBERS RESULTS:")
    print(f"{'='*80}")
    print(f"\nAverage Effect of Individual LP Numbers:")
    print(f"  {DEFAULT_BIAS_TOKEN}: {individual_lp_df['bias_prob'].mean():.8f} ({individual_lp_df['bias_ratio'].mean():.3f}x)")
    print(f"  {DEFAULT_CONTROL_TOKEN}: {individual_lp_df['control_prob'].mean():.8f} ({individual_lp_df['control_ratio'].mean():.3f}x)")

    print(f"\nEffect of All LP Numbers Combined:")
    print(f"  {DEFAULT_BIAS_TOKEN}: {combined_lp['bias_prob']:.8f} ({combined_lp['bias_ratio']:.3f}x)")
    print(f"  {DEFAULT_CONTROL_TOKEN}: {combined_lp['control_prob']:.8f} ({combined_lp['control_ratio']:.3f}x)")

    print(f"\n{'='*80}")
    print("KEY FINDINGS:")
    print(f"{'='*80}")
    hp_bias_boost = individual_hp_df['bias_ratio'].mean()
    lp_bias_boost = individual_lp_df['bias_ratio'].mean()
    hp_control_boost = individual_hp_df['control_ratio'].mean()
    lp_control_boost = individual_lp_df['control_ratio'].mean()

    print(f"\nHP Numbers vs LP Numbers (Individual Average):")
    print(f"  {DEFAULT_BIAS_TOKEN} boost: {hp_bias_boost:.3f}x (HP) vs {lp_bias_boost:.3f}x (LP) | Diff: {hp_bias_boost - lp_bias_boost:.3f}x")
    print(f"  {DEFAULT_CONTROL_TOKEN} boost: {hp_control_boost:.3f}x (HP) vs {lp_control_boost:.3f}x (LP) | Diff: {hp_control_boost - lp_control_boost:.3f}x")

    print(f"\nHP Numbers vs LP Numbers (Combined):")
    print(f"  {DEFAULT_BIAS_TOKEN} boost: {combined_hp['bias_ratio']:.3f}x (HP) vs {combined_lp['bias_ratio']:.3f}x (LP) | Diff: {combined_hp['bias_ratio'] - combined_lp['bias_ratio']:.3f}x")
    print(f"  {DEFAULT_CONTROL_TOKEN} boost: {combined_hp['control_ratio']:.3f}x (HP) vs {combined_lp['control_ratio']:.3f}x (LP) | Diff: {combined_hp['control_ratio'] - combined_lp['control_ratio']:.3f}x")


    print("\n" + "=" * 80)
    print(" EXPERIMENT COMPLETE")
    print("=" * 80)

 SUBLIMINAL LEARNING EXPERIMENT - REVISED WORKFLOW
Loading gpt2-medium on cpu...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

...

Showing: individual_hp_lp_effects


Showing: ratio_comparison


Showing: summary_comparison


Showing: hp_vs_lp_comparison



 FINAL SUMMARY

Baseline Probabilities:
  cat: 0.00047573
  dog: 0.00085278

HIGH-PROBABILITY (HP) NUMBERS RESULTS:

Average Effect of Individual HP Numbers:
  cat: 0.00174845 (3.675x)
  dog: 0.00174287 (2.044x)

Effect of All HP Numbers Combined:
  cat: 0.00394377 (8.290x)
  dog: 0.00329586 (3.865x)

LOW-PROBABILITY (LP) NUMBERS RESULTS:

Average Effect of Individual LP Numbers:
  cat: 0.00166863 (3.508x)
  dog: 0.00240869 (2.824x)

Effect of All LP Numbers Combined:
  cat: 0.00221355 (4.653x)
  dog: 0.00284967 (3.342x)

KEY FINDINGS:

HP Numbers vs LP Numbers (Individual Average):
  cat boost: 3.675x (HP) vs 3.508x (LP) | Diff: 0.168x
  dog boost: 2.044x (HP) vs 2.824x (LP) | Diff: -0.781x

HP Numbers vs LP Numbers (Combined):
  cat boost: 8.290x (HP) vs 4.653x (LP) | Diff: 3.637x
  dog boost: 3.865x (HP) vs 3.342x (LP) | Diff: 0.523x

 EXPERIMENT COMPLETE
